In [7]:
import pandas as pd
import os

In [8]:
twenty_train = pd.read_csv('../data/20/train.csv')
thirty_train = pd.read_csv('../data/30/train.csv')
fifty_train = pd.read_csv('../data/50/train.csv')
hundred_train = pd.read_csv('../data/100/train.csv')
two_hundred_train = pd.read_csv('../data/200/train.csv')

In [9]:
# upon the original training data, add some more groups with the same accountids being in the same group, and make sure there's at least one anomaly in each group
for data, group_size in zip([twenty_train, thirty_train, fifty_train, hundred_train, two_hundred_train], [20, 30, 50, 100, 200]):
    has_anomalies = data.groupby('accountid')['anomaly'].sum() > 0
    has_anomalies_accountids = has_anomalies[has_anomalies].index
    new_qids = range(data['qid'].max() + 1, data['qid'].max() + 1 + len(has_anomalies_accountids))
    for qid, accountid in zip(new_qids, has_anomalies_accountids):
        # create new groups 
        new_group_df = data[data['accountid'] == accountid].copy()
        new_group_df['qid'] = qid
        # make sure it does not exceed the original group size, and there's at least one anomaly
        while new_group_df['anomaly'].sum() == 0:
            new_group_df = new_group_df.sample(min(group_size, len(new_group_df)))
        data = pd.concat([data, new_group_df])
    os.mkdir(f'../data/{group_size}_extend')
    data.to_csv(f'../data/{group_size}_extend/train.csv', index=False)
        